
# Agentic AI Workshop 1
# Part 2: Embeddings & Similarity Metrics (No Vector DB Yet)


WHAT YOU'LL LEARN IN THIS NOTEBOOK:

1. What are embeddings?
   - Converting text into numerical vectors (arrays of numbers)
   - Why embeddings capture semantic meaning, not just keywords
   - How similar meanings result in similar vectors

2. Similarity Metrics
   - Cosine Similarity: Measures the angle between vectors (most common)
   - Euclidean Similarity: Measures the distance between vectors
   - When to use each metric

3. Document Retrieval Without a Database
   - How to find the most relevant documents for a query
   - Why this is the foundation of RAG (Retrieval-Augmented Generation)
   - Understanding the difference between keyword search and semantic search


In [ ]:
import os
from openai import OpenAI
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('../utils')
from helpers import load_env

In [ ]:

# Load and verify API keys
load_env(api_key_type="OPENAI_API_KEY")

In [ ]:
# Outputs directory
OUTPUTS_DIR = './../outputs/'
os.makedirs(OUTPUTS_DIR, exist_ok=True)

In [ ]:
# Initialize OpenAI client
client = OpenAI()  # Uses OPENAI_API_KEY from environment
print("Client ready.")

In [ ]:

def get_embeddings(texts, model="text-embedding-3-small"):
    # Returns an array of shape (N, D)
    resp = client.embeddings.create(model=model, input=texts)
    return np.array([d.embedding for d in resp.data], dtype=np.float32)

In [ ]:
def cosine_similarity(a, b):
    """Cosine similarity: measures angle between vectors (range: -1 to 1)"""
    a = a / (np.linalg.norm(a) + 1e-8)
    b = b / (np.linalg.norm(b) + 1e-8)
    return float(np.dot(a, b))

def cosine_distance(a, b):
    """Cosine distance: 1 minus cosine similarity (range: 0 to 2)"""
    return 1 - cosine_similarity(a, b)

def euclidean_similarity(a, b):
    """Euclidean similarity: inverse of Euclidean distance (normalized to 0-1 range)"""
    distance = np.linalg.norm(a - b)
    # Convert distance to similarity (smaller distance = higher similarity)
    # Using exponential decay for better visualization
    similarity = np.exp(-distance / 10)
    return float(similarity)

# Dictionary of available similarity functions
SIMILARITY_METRICS = {
    "cosine": cosine_similarity,
    "euclidean": euclidean_similarity
}

In [ ]:
# helper function to plot results
def plot_similarity_scores(df, query, metric_name, save_filename=None):
    """
    Create a horizontal bar chart showing document similarity scores.
    
    Args:
        df: DataFrame with 'title' and 'similarity' columns (sorted descending)
        query: The query string used for comparison
        metric_name: Name of the similarity metric used
        save_filename: Optional filename to save the chart (without path)
    """
    plt.figure(figsize=(10, 6))
    bars = plt.barh(df["title"], df["similarity"], color='#1db1c1')
    plt.xlabel(f'{metric_name.title()} Similarity Score', fontsize=12)
    plt.ylabel('Document', fontsize=12)
    plt.title(f'Document Relevance to Query ({metric_name.title()} Similarity)\n"{query}"', 
              fontsize=14, pad=20)
    plt.xlim(0, 1)  # Normalized similarity range
    plt.gca().invert_yaxis()  # Highest score at top
    plt.grid(axis='x', alpha=0.3)

    # Add score labels at the end of each bar
    for bar, score in zip(bars, df["similarity"]):
        plt.text(bar.get_width() + 0.01,  # x position (slightly after bar end)
                 bar.get_y() + bar.get_height()/2,  # y position (center of bar)
                 f'{score:.3f}',  # score with 3 decimal places
                 va='center',  # vertical alignment
                 fontsize=10,
                 fontweight='bold')

    plt.tight_layout()

    # Save the chart if filename provided
    if save_filename:
        chart_path = os.path.join(OUTPUTS_DIR, save_filename)
        plt.savefig(chart_path, dpi=150, bbox_inches='tight')
        print(f"Chart saved to: {chart_path}")

    plt.show()

In [ ]:
# Documents with more distinct topics
# Normally, you would read these from some knowledge base (docs can be pdfs, markdown, txt, etc.)
documents = [
    {
        "title": "Deep Learning",
        "content": "Deep learning uses neural networks with multiple layers to learn hierarchical representations from data. It excels at tasks like image recognition and natural language processing."
    },
    {
        "title": "Time Series Forecasting",
        "content": "Time series forecasting estimates future values based on historical patterns. Common methods include ARIMA, exponential smoothing, and recurrent neural networks for sequential data."
    },
    {
        "title": "Model Deployment",
        "content": "Model deployment involves packaging trained models into production systems. This includes creating APIs, containerization with Docker, and monitoring model performance in real-time."
    },
    {
        "title": "Feature Engineering",
        "content": "Feature engineering transforms raw data into meaningful inputs for machine learning models. Techniques include scaling, encoding categorical variables, and creating interaction terms."
    },
    {
        "title": "Anomaly Detection",
        "content": "Anomaly detection identifies unusual patterns that deviate from normal behavior. Applications include fraud detection, network security, and quality control in manufacturing."
    }
]

In [ ]:
# CONFIGURATION: Choose similarity metric
# Options: ""cosine or "euclidean"
SIMILARITY_METRIC = "cosine"  # Change this to "euclidean" to try different metric

# Get the similarity function
similarity_func = SIMILARITY_METRICS[SIMILARITY_METRIC]


In [ ]:
# EXAMPLE 1: Query about deployment

query1 = "What's the best way to put my trained model into production?"

print("\n" + "="*80)
print(f"EXAMPLE 1 - Using {SIMILARITY_METRIC.upper()} similarity")
print("="*80)
print(f"QUERY: {query1}")
print("="*80)

# Extract content for embeddings
doc_contents = [doc["content"] for doc in documents]
doc_titles = [doc["title"] for doc in documents]

# Get embeddings for Example 1
doc_embeddings = get_embeddings(doc_contents)
query1_embedding = get_embeddings([query1])[0]

# Compute similarities
scores1 = [similarity_func(query1_embedding, e) for e in doc_embeddings]

# Create dataframe
df1 = pd.DataFrame({
    "title": doc_titles,
    "document": doc_contents,
    "similarity": scores1
}).sort_values("similarity", ascending=False)

print(f"\nDocument Similarity Scores using {SIMILARITY_METRIC.upper()} (sorted by relevance):\n")
print(df1[["title", "similarity"]].to_string(index=False))
print("\n" + "="*80)

# Plot Example 1
plot_similarity_scores(df1, query1, SIMILARITY_METRIC, 
                      save_filename=f'similarity_example1_{SIMILARITY_METRIC}.png')


In [ ]:
# EXAMPLE 2: Query about temporal data

query2 = "How can I predict stock prices for the next quarter?"

print("\n" + "="*80)
print(f"EXAMPLE 2 - Using {SIMILARITY_METRIC.upper()} similarity")
print("="*80)
print(f"QUERY: {query2}")
print("="*80)

# Get embeddings for Example 2 (reuse doc_embeddings)
query2_embedding = get_embeddings([query2])[0]

# Compute similarities
scores2 = [similarity_func(query2_embedding, e) for e in doc_embeddings]

# Create dataframe
df2 = pd.DataFrame({
    "title": doc_titles,
    "document": doc_contents,
    "similarity": scores2
}).sort_values("similarity", ascending=False)

print(f"\nDocument Similarity Scores using {SIMILARITY_METRIC.upper()} (sorted by relevance):\n")
print(df2[["title", "similarity"]].to_string(index=False))
print("\n" + "="*80)

# Plot Example 2
plot_similarity_scores(df2, query2, SIMILARITY_METRIC, 
                      save_filename=f'similarity_example2_{SIMILARITY_METRIC}.png')

# Exercises

### EXERCISE 1: Experiment with Different Queries (Beginner)
----------------------------------------------------------
Try these queries and observe which documents rank highest:

1. "How do I prepare my data before training a model?"
2. "What techniques help identify fraudulent transactions?"
3. "I want to forecast sales for the next month"

### EXERCISE 2: Add Your Own Documents (Intermediate)
----------------------------------------------------------
Add 2-3 more documents to the collection on AI/ML topics of your choice.
Examples: "Reinforcement Learning", "Natural Language Processing", "Computer Vision"

Then create a query that should match YOUR new documents best.


### EXERCISE 3: Compare Different Similarity Metrics (Intermediate)
----------------------------------------------------------
We've been using cosine similarity, but there are other ways to measure similarity!

Other metrics to try:
- "dot_product": Direct multiplication of vectors (fast!)
- "manhattan": Sum of absolute differences (robust to outliers)

Your tasks:
1. Implement and try other metrics and analyze the differences in results
2. Re-run both examples and observe the results
3. Document your findings:
